Week 2

In [ ]:
pip install geocoder

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import geocoder

In [2]:
url = 'https://en.wikipedia.org/wiki/Boroughs_of_Amsterdam'
wiki_page = requests.get(url)
url

'https://en.wikipedia.org/wiki/Boroughs_of_Amsterdam'

In [3]:
soup = BeautifulSoup(wiki_page.content, 'html.parser')
table = soup.find('table', {'class':'wikitable sortable'}).tbody
rows = table.find_all('tr')
columns = [i.text.replace('\n', '') for i in rows[0].find_all('th')]
df_1 = pd.DataFrame(columns = columns)
col = ["Borough","Neighbourhoods"]
df_a = pd.DataFrame(columns = col)
df_1

,Borough,Area,Population,Population density,Location (in green),Neighbourhoods


In [4]:

for i in range(1, len(rows)):
    tds = rows[i].find_all('td')
    data = []
    bor = ""
    neigh = []
    for z in range(0, len(tds)):
        values = tds[z].text.replace("\n","")
        data.append(values)
        if (z==0):
            bor = tds[z].text
        if (z==len(tds)-1):
            neigh = tds[z].text.split(",")
    for x in neigh:
        df_a = df_a.append(pd.Series([bor,x.replace("\n","")], index=col), ignore_index = True)
    df_1 = df_1.append(pd.Series(data, index=columns), ignore_index = True)
df_1
df_a

,Borough,Neighbourhoods
0,Centrum (Centre),Binnenstad
1,Centrum (Centre),Grachtengordel
2,Centrum (Centre),Haarlemmerbuurt
3,Centrum (Centre),Jodenbuurt
4,Centrum (Centre),Jordaan
...,...,...
64,Zuid (South),Zuidas
65,Zuidoost(Southeast),Bijlmermeer
66,Zuidoost(Southeast),Venserpolder
67,Zuidoost(Southeast),Gaasperdam


In [5]:
def get_latlng(arcgis_geocoder):
    
    lat_lng_coords = None
    
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Amsterdam, Netherlands'.format(arcgis_geocoder))
        lat_lng_coords = g.latlng
    return lat_lng_coords

districts = df_1['Borough']
coordinates = [get_latlng(districts) for districts in districts.tolist()]
districts

0           Centrum (Centre)
1              Noord (North)
2       Nieuw-West(New West)
3                Oost (East)
4                       West
5    Westpoort(West Gateway)
6               Zuid (South)
7        Zuidoost(Southeast)
Name: Borough, dtype: object

In [6]:
neighbourhood = df_a['Neighbourhoods']
coordinates_nei = [get_latlng(neighbourhood) for neighbourhood in neighbourhood.tolist()]
print(neighbourhood)

0           Binnenstad
1       Grachtengordel
2      Haarlemmerbuurt
3           Jodenbuurt
4              Jordaan
            ...       
64              Zuidas
65         Bijlmermeer
66        Venserpolder
67          Gaasperdam
68            Driemond
Name: Neighbourhoods, Length: 69, dtype: object


In [7]:
df = df_1

df_2 = pd.DataFrame(coordinates, columns = ['Latitude', 'Longitude'])
df_2

,Latitude,Longitude
0,52.407775,4.935950
1,52.381501,4.916138
2,52.369930,4.907880
3,52.360130,4.925320
4,52.372800,4.889760
5,52.402710,4.837610
6,52.376380,4.937500
7,52.369930,4.907880


In [8]:
df_a2 = pd.DataFrame(coordinates_nei, columns = ['Latitude', 'Longitude'])
df_a['Latitude']=df_a2['Latitude']
df_a['Longitude']=df_a2['Longitude']
df_a

,Borough,Neighbourhoods,Latitude,Longitude
0,Centrum (Centre),Binnenstad,52.372160,4.904370
1,Centrum (Centre),Grachtengordel,52.369930,4.907880
2,Centrum (Centre),Haarlemmerbuurt,52.385031,4.852415
3,Centrum (Centre),Jodenbuurt,52.363000,4.884360
4,Centrum (Centre),Jordaan,52.369520,4.880910
...,...,...,...,...
64,Zuid (South),Zuidas,52.336690,4.875920
65,Zuidoost(Southeast),Bijlmermeer,52.307341,4.969019
66,Zuidoost(Southeast),Venserpolder,52.326730,4.945590
67,Zuidoost(Southeast),Gaasperdam,52.296770,4.978260


In [9]:
df['Latitude']=df_2['Latitude']
df['Longitude']=df_2['Longitude']
df

,Borough,Area,Population,Population density,Location (in green),Neighbourhoods,Latitude,Longitude
0,Centrum (Centre),8.04 km²,"86,422","13,748/km²",,"Binnenstad, Grachtengordel, Haarlemmerbuurt, J...",52.407775,4.935950
1,Noord (North),49.01 km²,"94,766","2,269/km²",,"Banne Buiksloot, Buiksloot, Buikslotermeer, Fl...",52.381501,4.916138
2,Nieuw-West(New West),32.38 km²,"151,677","4,478/km²",,"Geuzenveld, Nieuw Sloten, Oostoever, Osdorp, O...",52.369930,4.907880
3,Oost (East),30.56 km²,"135,767","7,635/km²",,"IJburg, Indische Buurt, Eastern Docklands, Oud...",52.360130,4.925320
4,West,9.89 km²,"143,842","15,252/km²",,"Frederik Hendrikbuurt, Houthaven, Spaarndammer...",52.372800,4.889760
5,Westpoort(West Gateway),10 km²,192,10/km²,,Westpoort,52.402710,4.837610
6,Zuid (South),17.41 km²,"144,432","9,349/km²",,"Apollobuurt, Buitenveldert, Hoofddorppleinbuur...",52.376380,4.937500
7,Zuidoost(Southeast),22.08 km²,"87,854","4,391/km²",,"Bijlmermeer, Venserpolder, Gaasperdam, Driemond",52.369930,4.907880


pip install folium

In [10]:
pip install KMeans

/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Note: you may need to restart the kernel to use updated packages.


In [11]:
import numpy as np
import json 
from geopy.geocoders import Nominatim 

import requests 
from pandas import json_normalize

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans
import folium

In [12]:
df_a3 = pd.DataFrame(columns = ["No Venues"])
df_a3

,No Venues


In [13]:
LIMIT = 100
radius = 500
CLIENT_ID = "LSI4RRKORY0L2TMLE0MSPOHUWXOJO2TYR1EU35LSC0NAVGEH"
CLIENT_SECRET = "4J23HSGCVOP13DKTE443XT5GBLGZUHAUMGSXJTNLHTSC3V3G"
VERSION = "20210725"
for y in range(0, df_a.shape[0]):
    neighborhood_latitude = df_a.loc[y,"Latitude"]
    neighborhood_longitude = df_a.loc[y, "Longitude"]
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&query=asian&categoryId=4bf58dd8d48988d142941735'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        neighborhood_latitude, 
        neighborhood_longitude, 
        radius, 
        LIMIT)
    results = requests.get(url).json()
    nearby_venues = json_normalize(results['response']['groups'][0]['items'])
    df_a3 = df_a3.append(pd.Series([nearby_venues.shape[0]], index=["No Venues"]), ignore_index = True)
df_a3

,No Venues
0,43
1,7
2,7
3,24
4,17
...,...
64,10
65,0
66,0
67,2


In [14]:
df_a["No Venues"] = df_a3["No Venues"] 

In [15]:
df_a

,Borough,Neighbourhoods,Latitude,Longitude,No Venues
0,Centrum (Centre),Binnenstad,52.372160,4.904370,43
1,Centrum (Centre),Grachtengordel,52.369930,4.907880,7
2,Centrum (Centre),Haarlemmerbuurt,52.385031,4.852415,7
3,Centrum (Centre),Jodenbuurt,52.363000,4.884360,24
4,Centrum (Centre),Jordaan,52.369520,4.880910,17
...,...,...,...,...,...
64,Zuid (South),Zuidas,52.336690,4.875920,10
65,Zuidoost(Southeast),Bijlmermeer,52.307341,4.969019,0
66,Zuidoost(Southeast),Venserpolder,52.326730,4.945590,0
67,Zuidoost(Southeast),Gaasperdam,52.296770,4.978260,2


ams_clustering = df_a.drop(['Borough','Neighbourhoods','No Venues'],1)
ams_clustering

In [17]:
df_ak = df_a
df_an = df_a
df_a.sort_values(by=['No Venues'], inplace=True, ascending=False)
df_an = df_a.head(50)
ams_clustering = df_an.drop(['Borough','Neighbourhoods','No Venues'],1)
df_an

,Borough,Neighbourhoods,Latitude,Longitude,No Venues
0,Centrum (Centre),Binnenstad,52.372160,4.904370,43
57,Zuid (South),De Pijp,52.356250,4.890570,41
13,Centrum (Centre),Weteringschans,52.361751,4.885341,27
62,Zuid (South),Vondelpark,52.361800,4.881720,27
3,Centrum (Centre),Jodenbuurt,52.363000,4.884360,24
11,Centrum (Centre),Uilenburg,52.369363,4.902743,18
4,Centrum (Centre),Jordaan,52.369520,4.880910,17
55,Zuid (South),Hoofddorppleinbuurt,52.351597,4.850200,17
60,Zuid (South),Schinkelbuurt,52.351657,4.852859,16
39,West,Frederik Hendrikbuurt,52.378645,4.877720,14


In [18]:
ams_clustering

,Latitude,Longitude
0,52.372160,4.904370
57,52.356250,4.890570
13,52.361751,4.885341
62,52.361800,4.881720
3,52.363000,4.884360
11,52.369363,4.902743
4,52.369520,4.880910
55,52.351597,4.850200
60,52.351657,4.852859
39,52.378645,4.877720


In [19]:

k=5
kmeans = KMeans(n_clusters = k,random_state=0).fit(ams_clustering)
kmeans.labels_
df_an.insert(0, 'Cluster Labels', kmeans.labels_)
df_an

,Cluster Labels,Borough,Neighbourhoods,Latitude,Longitude,No Venues
0,0,Centrum (Centre),Binnenstad,52.372160,4.904370,43
57,2,Zuid (South),De Pijp,52.356250,4.890570,41
13,2,Centrum (Centre),Weteringschans,52.361751,4.885341,27
62,2,Zuid (South),Vondelpark,52.361800,4.881720,27
3,2,Centrum (Centre),Jodenbuurt,52.363000,4.884360,24
11,0,Centrum (Centre),Uilenburg,52.369363,4.902743,18
4,2,Centrum (Centre),Jordaan,52.369520,4.880910,17
55,2,Zuid (South),Hoofddorppleinbuurt,52.351597,4.850200,17
60,2,Zuid (South),Schinkelbuurt,52.351657,4.852859,16
39,2,West,Frederik Hendrikbuurt,52.378645,4.877720,14


In [29]:
df_ah = df_a
df_a.sort_values(by=['No Venues'], inplace=True, ascending=False)
df_ah = df_a.head(50)
ams_clustering2= df_ah.drop(['Borough','Neighbourhoods'],1)
ams_clustering2
k=5
kmeans2 = KMeans(n_clusters = k,random_state=0).fit(ams_clustering2)
kmeans2.labels_

df_ah.insert(0, 'Cluster Labels', kmeans2.labels_)
df_ah

,Cluster Labels,Borough,Neighbourhoods,Latitude,Longitude,No Venues
0,1,Centrum (Centre),Binnenstad,52.372160,4.904370,43
57,1,Zuid (South),De Pijp,52.356250,4.890570,41
13,4,Centrum (Centre),Weteringschans,52.361751,4.885341,27
62,4,Zuid (South),Vondelpark,52.361800,4.881720,27
3,4,Centrum (Centre),Jodenbuurt,52.363000,4.884360,24
11,2,Centrum (Centre),Uilenburg,52.369363,4.902743,18
4,2,Centrum (Centre),Jordaan,52.369520,4.880910,17
55,2,Zuid (South),Hoofddorppleinbuurt,52.351597,4.850200,17
60,2,Zuid (South),Schinkelbuurt,52.351657,4.852859,16
39,2,West,Frederik Hendrikbuurt,52.378645,4.877720,14


In [20]:
# create map
map_clusters = folium.Map(location=[52.372160,4.904370],zoom_start=10)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, neighbourhood, cluster in zip(df_an['Latitude'], df_an['Longitude'], df_an['Neighbourhoods'], df_an['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [30]:
# create map
map_clusters1 = folium.Map(location=[52.372160,4.904370],zoom_start=10)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, neighbourhood, cluster in zip(df_ah['Latitude'], df_ah['Longitude'], df_ah['Neighbourhoods'], df_ah['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters1)
       
map_clusters1